In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv
/kaggle/input/tabular-playground-series-apr-2022/train.csv
/kaggle/input/tabular-playground-series-apr-2022/test.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
df_train_labels = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')

In [3]:
df_averaged = df_train.groupby(['sequence','subject']).mean().reset_index()
df_averaged = df_averaged.drop(labels = ['sequence','step'],axis=1)
df_averaged.columns = [''.join(str(col)).strip()+'_avg' for col in df_averaged.columns.values]
df_averaged.head()


df_std = df_train.groupby(['sequence','subject']).std().reset_index()
df_std = df_std.drop(labels = ['sequence','step'],axis=1)
df_std.columns = [''.join(str(col)).strip()+'_std' for col in df_std.columns.values]
df_std.head()


df_max = df_train.groupby(['sequence','subject']).max().reset_index()
df_max = df_max.drop(labels = ['sequence','step'],axis=1)
df_max.columns = [''.join(str(col)).strip()+'_max' for col in df_max.columns.values]
df_max.head()

df_min = df_train.groupby(['sequence','subject']).min().reset_index()
df_min = df_min.drop(labels = ['sequence','step'],axis=1)
df_min.columns = [''.join(str(col)).strip()+'_min' for col in df_min.columns.values]
df_min.head()


df_fin = pd.concat([df_averaged,df_std,df_min,df_max],axis=1)
df_fin = df_fin.drop(labels = ['subject_std','subject_min','subject_max'],axis=1)
df_fin.head()



,subject_avg,sensor_00_avg,sensor_01_avg,sensor_02_avg,sensor_03_avg,sensor_04_avg,sensor_05_avg,sensor_06_avg,sensor_07_avg,sensor_08_avg,...,sensor_03_max,sensor_04_max,sensor_05_max,sensor_06_max,sensor_07_max,sensor_08_max,sensor_09_max,sensor_10_max,sensor_11_max,sensor_12_max
0,47,0.041744,-0.013025,0.739306,0.034204,-0.054774,0.018537,0.016545,0.055153,0.031667,...,5.712389,0.971886,1.280479,2.977231,9.193893,2.3,1.964595,1.809610,2.989712,6.674766
1,66,-0.069294,-0.019958,-1.602773,0.120162,0.147499,-0.002032,-0.053780,-0.009746,-0.075000,...,5.637168,3.661950,5.956619,2.775046,6.950382,2.3,3.800578,9.955438,8.044239,144.598039
2,66,-0.001546,0.025000,-1.008764,-0.024794,-0.174372,-0.000362,0.000926,0.162150,0.010000,...,2.827434,5.368985,6.344054,3.257741,6.484733,4.6,2.854769,8.387898,1.475309,1.116795
3,542,0.029869,-0.049177,0.494766,0.033481,-0.018023,0.002344,0.011187,0.046336,-0.045000,...,5.435398,0.543682,2.423336,3.170310,7.361069,5.0,8.065029,1.586626,3.593621,0.593350
4,437,0.059132,0.014916,0.900000,-0.056268,0.004291,-0.006644,0.056239,-0.004847,-0.055000,...,3.365487,1.485148,4.103964,5.591075,3.423664,2.8,2.784682,0.633599,6.989712,0.135976


In [4]:
target = df_train_labels['state']
input_matrix = df_fin
from sklearn.impute import SimpleImputer
imp=SimpleImputer( missing_values=np.nan,strategy='mean',fill_value=None,verbose=0,copy=True )
input_matrix=imp.fit_transform(input_matrix)
from sklearn import preprocessing
input_matrix= preprocessing.StandardScaler().fit(input_matrix).transform(input_matrix)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
#Split Data for training and testing
X_train, X_test, y_train, y_test = train_test_split(input_matrix, target,test_size=0.33)

from sklearn import svm

#Finding best kernel
knls = ['linear', 'rbf', 'sigmoid']
acc_svm = []
for kernel in knls:
    clf_krnl = svm.SVC(kernel=kernel,gamma="auto").fit(X_train, y_train)
    svm_predict = clf_krnl.predict(X_test)
    acc_svm.append(metrics.accuracy_score(y_test, svm_predict)) 
    
print("Maximum accuracy:",max(acc_svm),"with Kernel =",knls[acc_svm.index(max(acc_svm))])

Maximum accuracy: 0.7698949824970829 with Kernel = rbf


In [6]:
#Training the model with best kernel 
kernel="rbf"
clf_krnl = svm.SVC(kernel=kernel,gamma="auto").fit(X_train, y_train)

In [7]:
df_averaged = df_test.groupby(['sequence','subject']).mean().reset_index()
df_averaged = df_averaged.drop(labels = ['sequence','step'],axis=1)
df_averaged.columns = [''.join(str(col)).strip()+'_avg' for col in df_averaged.columns.values]
df_averaged.head()


df_std = df_test.groupby(['sequence','subject']).std().reset_index()
df_std = df_std.drop(labels = ['sequence','step'],axis=1)
df_std.columns = [''.join(str(col)).strip()+'_std' for col in df_std.columns.values]
df_std.head()


df_max = df_test.groupby(['sequence','subject']).max().reset_index()
df_max = df_max.drop(labels = ['sequence','step'],axis=1)
df_max.columns = [''.join(str(col)).strip()+'_max' for col in df_max.columns.values]
df_max.head()

df_min = df_test.groupby(['sequence','subject']).min().reset_index()
df_min = df_min.drop(labels = ['sequence','step'],axis=1)
df_min.columns = [''.join(str(col)).strip()+'_min' for col in df_min.columns.values]
df_min.head()


df_fin = pd.concat([df_averaged,df_std,df_min,df_max],axis=1)
df_fin = df_fin.drop(labels = ['subject_std','subject_min','subject_max'],axis=1)
df_fin.head()

input_test_matrix = df_fin
from sklearn.impute import SimpleImputer
imp=SimpleImputer( missing_values=np.nan,strategy='mean',fill_value=None,verbose=0,copy=True )
input_test_matrix=imp.fit_transform(input_test_matrix)
from sklearn import preprocessing
input_test_matrix= preprocessing.StandardScaler().fit(input_test_matrix).transform(input_test_matrix)



In [8]:
yhat_svm = clf_krnl.predict(input_test_matrix)

In [9]:
import csv
with open('submission.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['sequence', 'state'])
    for i,v in enumerate(yhat_svm):
        csvwriter.writerow([i+25968,str(v)])